In [1]:
import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

In [2]:
import datajoint as dj
from datetime import datetime
import spikeinterface as si
from spikeinterface import widgets, exporters, postprocessing, qualitymetrics, sorters
from workflow.pipeline import *
from workflow.utils.paths import (
    get_ephys_root_data_dir,
    get_raw_root_data_dir,
    get_processed_root_data_dir,
)
from element_interface.utils import dict_to_uuid, find_full_path, find_root_directory

[2024-06-04 23:44:34,609][INFO]: Connecting milagros@db.datajoint.com:3306
[2024-06-04 23:44:36,152][INFO]: Connected milagros@db.datajoint.com:3306


### Select from the following list of experiments


In [3]:
display(
    culture.Experiment()
    .proj("experiment_end_time", "drug_name", "drug_concentration", "experiment_plan")
    .fetch(format="frame")
    .reset_index()
)

,organoid_id,experiment_start_time,experiment_end_time,drug_name,drug_concentration,experiment_plan
0,O09,2023-05-18 12:25:00,2023-05-18 18:15:00,4-AP,100.0,ephys
1,O09,2023-05-18 18:15:00,2023-05-19 09:30:00,No Drug,NaN,ephys
2,O09,2023-05-19 09:30:00,2023-05-19 15:35:00,Bicuculline,50.0,ephys
3,O09,2023-05-19 15:45:00,2023-05-20 15:40:00,Tetrodotoxin,1.0,ephys
4,O10,2023-05-18 12:25:00,2023-05-18 18:15:00,4-AP,100.0,ephys
5,O10,2023-05-18 18:15:00,2023-05-19 09:30:00,No Drug,NaN,ephys
6,O10,2023-05-19 09:30:00,2023-05-19 15:35:00,Bicuculline,50.0,ephys
7,O10,2023-05-19 15:45:00,2023-05-20 15:40:00,Tetrodotoxin,1.0,ephys
8,O11,2023-05-18 12:25:00,2023-05-18 18:15:00,4-AP,100.0,ephys
9,O11,2023-05-18 18:15:00,2023-05-19 09:30:00,No Drug,NaN,ephys


### Create `spike_sorting` sessions


In [4]:
session_info = dict(
    organoid_id="O09",
    experiment_start_time="2023-05-18 12:25:00",
    insertion_number=0,
    start_time="2023-05-18 12:25:00",
    end_time="2023-05-18 12:30:00",
    session_type="spike_sorting",
)

session_probe_info = dict(
    organoid_id="O09",
    experiment_start_time="2023-05-18 12:25:00",
    insertion_number=0,
    start_time="2023-05-18 12:25:00",
    end_time="2023-05-18 12:30:00",
    probe="Q983",  # probe serial number
    port_id="A",  # Port ID ("A", "B", etc.)
    used_electrodes=[],  # empty if all electrodes were used
)

In [5]:
# Insert the session
SPIKE_SORTING_DURATION = 120  # seconds

# Start and end time of the session. It should be within the experiment time range
start_time = datetime.strptime(session_info["start_time"], "%Y-%m-%d %H:%M:%S")
end_time = datetime.strptime(session_info["end_time"], "%Y-%m-%d %H:%M:%S")
duration = (end_time - start_time).total_seconds() / 60  # minutes

assert (
    session_info["session_type"] == "spike_sorting"
    and duration <= SPIKE_SORTING_DURATION
), f"Session type must be 'spike_sorting' and duration must be less than {SPIKE_SORTING_DURATION} minutes"

ephys.EphysSession.insert1(session_info, ignore_extra_fields=True, skip_duplicates=True)

ephys.EphysSessionProbe.insert1(
    session_probe_info, ignore_extra_fields=True, skip_duplicates=True
)

del session_probe_info["used_electrodes"]
display(ephys.EphysSession & session_info)
display(ephys.EphysSessionProbe & session_probe_info)

key = (ephys.EphysSession & session_info).fetch1("KEY")

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,session_type
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,spike_sorting


organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,probe unique identifier for this model of probe (e.g. serial number),port_id,"used_electrodes list of electrode IDs used in this session (if null, all electrodes are used)"
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,Q983,A,=BLOB=


### Insert clustering parameters


#### Sample paramter dictionary. It expects to have `SI_SORTING_PARAMS`, `SI_PREPROCESSING_METHOD`, `SI_QUALITY_METRICS_PARAMS`, `SI_JOB_KWARGS`

- `SI_SORTING_PARAMS`: Run `si.sorters.get_default_sorter_params(sorter_name)` to get the default parameter for a sorter. Modify values if needed. If empty, the sorter will be run with the default parameter.

- `SI_PREPROCESSING_METHOD`: Select a preprocesesing function from `si_preprocessing.py`
- `SI_WAVEFORM_EXTRACTION_PARAMS`: Waveform extraction parameters. If empty, the sorter will use the default parameter.
- `SI_QUALITY_METRICS_PARAMS`: Quality metric parameters. If empty, the sorter will use the default parameter.
- `SI_JOB_KWARGS`: Sorter job parameters. If empty, the sorter will use the default parameter.


In [6]:
ephys_sorter.SI_SORTERS  # list of available sorters supported by spikeinterface

['combinato',
 'hdsort',
 'herdingspikes',
 'ironclust',
 'kilosort',
 'kilosort2',
 'kilosort2.5',
 'kilosort3',
 'kilosort4',
 'pykilosort',
 'klusta',
 'mountainsort4',
 'mountainsort5',
 'spykingcircus',
 'tridesclous',
 'waveclus',
 'waveclus.snippets',
 'yass',
 'spykingcircus2',
 'tridesclous2',
 'simple']

- Print out the default parameter for a sorter.


In [7]:
sorter_name = "spykingcircus2"
si.sorters.get_default_sorter_params(sorter_name)

{'general': {'ms_before': 2, 'ms_after': 2, 'radius_um': 100},
 'sparsity': {'method': 'ptp', 'threshold': 0.25},
 'filtering': {'freq_min': 150,
  'freq_max': 7000,
  'ftype': 'bessel',
  'filter_order': 2},
 'detection': {'peak_sign': 'neg', 'detect_threshold': 4},
 'selection': {'method': 'uniform',
  'n_peaks_per_channel': 5000,
  'min_n_peaks': 100000,
  'select_per_channel': False,
  'seed': 42},
 'apply_motion_correction': True,
 'motion_correction': {'preset': 'nonrigid_fast_and_accurate'},
 'merging': {'minimum_spikes': 10,
  'corr_diff_thresh': 0.5,
  'template_metric': 'cosine',
  'censor_correlograms_ms': 0.4,
  'num_channels': None},
 'clustering': {'legacy': True},
 'matching': {'method': 'wobble'},
 'apply_preprocessing': True,
 'matched_filtering': True,
 'cache_preprocessing': {'mode': 'memory',
  'memory_limit': 0.5,
  'delete_cache': True},
 'multi_units_only': False,
 'job_kwargs': {'n_jobs': 0.8},
 'debug': False}

- Create a parameter dictionary


In [8]:
params = {}
params["SI_PREPROCESSING_METHOD"] = "organoid_preprocessing"
params["SI_SORTING_PARAMS"] = {
    "general": {"ms_before": 2, "ms_after": 2, "radius_um": 100},
    "filtering": {"freq_min": 150},
    "detection": {"peak_sign": "neg", "detect_threshold": 4},
    "selection": {
        "method": "smart_sampling_amplitudes",
        "n_peaks_per_channel": 5000,
        "min_n_peaks": 20000,
        "select_per_channel": False,
    },
    "clustering": {"legacy": False},
    "matching": {"method": "circus-omp-svd", "method_kwargs": {}},
    "apply_preprocessing": True,
    "cache_preprocessing": {
        "mode": "memory",
        "memory_limit": 0.5,
        "delete_cache": True,
    },
    "multi_units_only": False,
    "job_kwargs": {"n_jobs": 0.8},
    "debug": False,
}
params["SI_POSTPROCESSING_PARAMS"] = {
    "extensions": {
        "random_spikes": {},
        "waveforms": {},
        "templates": {},
        "noise_levels": {},
        # "amplitude_scalings": {},
        "correlograms": {},
        "isi_histograms": {},
        "principal_components": {"n_components": 5, "mode": "by_channel_local"},
        "spike_amplitudes": {},
        "spike_locations": {},
        "template_metrics": {"include_multi_channel_metrics": True},
        "template_similarity": {},
        "unit_locations": {},
        "quality_metrics": {},
    },
    "job_kwargs": {"n_jobs": -1, "chunk_duration": "1s"},
    "export_to_phy": True,
    "export_report": True,
}

- Insert the paramter. Specify `clustering_method (select from above)`, `paramset_desc (optional)`, `paramset_idx (int)`


In [9]:
paramset_idx = 1
clustering_method = "spykingcircus2"
paramset_desc = (
    "Default parameter set for spyking circus2 using SpikeInterface v0.101.*"
)

ephys.ClusteringParamSet.insert_new_params(
    paramset_idx=1,
    clustering_method=clustering_method,
    paramset_desc=paramset_desc,
    params=params,
)

### Select a session and `paramset_idx` and insert into `ephys.ClusteringTask` to trigger spike-sorting


In [11]:
clustering_task = key | {"paramset_idx": paramset_idx}

ephys.ClusteringTask.insert1(clustering_task)

ephys.ClusteringTask & key

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,paramset_idx,clustering_output_dir clustering output directory relative to the clustering root data directory
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,O09-12_raw/202305181225_202305181230/O09/spykingcircus2_0
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,1,O09-12_raw/202305181225_202305181230/O09/spykingcircus2_1


### Wait until spike-sorting is finished. Explore spike-sorting results in downstream tables


In [12]:
ephys.CuratedClustering.Unit & key

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,paramset_idx,unit,electrode_config_hash,probe_type e.g. A1x32-6mm-100-177-H32_21mm,"electrode electrode index, starts at 0","cluster_quality_label cluster quality type - e.g. 'good', 'MUA', 'noise', etc.",spike_count how many spikes in this recording for this unit,"spike_times (s) spike times of this unit, relative to the start of the EphysRecording",spike_sites array of electrode associated with each spike,"spike_depths (um) array of depths associated with each spike, relative to the (0, 0) of the probe"
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,0,699af5e0-31fa-acc9-1aeb-132c6972d25e,A1x32-6mm-100-177-H32_21mm,17,n.a.,2271,=BLOB=,=BLOB=,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,1,699af5e0-31fa-acc9-1aeb-132c6972d25e,A1x32-6mm-100-177-H32_21mm,1,n.a.,14434,=BLOB=,=BLOB=,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,2,699af5e0-31fa-acc9-1aeb-132c6972d25e,A1x32-6mm-100-177-H32_21mm,4,n.a.,12488,=BLOB=,=BLOB=,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,3,699af5e0-31fa-acc9-1aeb-132c6972d25e,A1x32-6mm-100-177-H32_21mm,18,n.a.,4674,=BLOB=,=BLOB=,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,4,699af5e0-31fa-acc9-1aeb-132c6972d25e,A1x32-6mm-100-177-H32_21mm,2,n.a.,14792,=BLOB=,=BLOB=,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,5,699af5e0-31fa-acc9-1aeb-132c6972d25e,A1x32-6mm-100-177-H32_21mm,14,n.a.,4410,=BLOB=,=BLOB=,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,6,699af5e0-31fa-acc9-1aeb-132c6972d25e,A1x32-6mm-100-177-H32_21mm,25,n.a.,11202,=BLOB=,=BLOB=,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,7,699af5e0-31fa-acc9-1aeb-132c6972d25e,A1x32-6mm-100-177-H32_21mm,3,n.a.,19064,=BLOB=,=BLOB=,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,8,699af5e0-31fa-acc9-1aeb-132c6972d25e,A1x32-6mm-100-177-H32_21mm,31,n.a.,7937,=BLOB=,=BLOB=,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,9,699af5e0-31fa-acc9-1aeb-132c6972d25e,A1x32-6mm-100-177-H32_21mm,16,n.a.,1348,=BLOB=,=BLOB=,=BLOB=


In [13]:
ephys.WaveformSet.PeakWaveform & key

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,paramset_idx,unit,peak_electrode_waveform (uV) mean waveform for a given unit at its representative electrode
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,0,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,1,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,2,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,3,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,4,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,5,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,6,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,7,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,8,=BLOB=
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,9,=BLOB=


In [14]:
ephys.QualityMetrics.Cluster & key

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,paramset_idx,unit,firing_rate (Hz) firing rate for a unit,snr signal-to-noise ratio for a unit,presence_ratio fraction of time in which spikes are present,isi_violation rate of ISI violation as a fraction of overall rate,number_violation total number of ISI violations,amplitude_cutoff estimate of miss rate based on amplitude histogram,isolation_distance distance to nearest cluster in Mahalanobis space,l_ratio,d_prime Classification accuracy based on LDA,nn_hit_rate Fraction of neighbors for target cluster that are also in target cluster,nn_miss_rate Fraction of neighbors outside target cluster that are in target cluster,silhouette_score Standard metric for cluster overlap,max_drift Maximum change in spike depth throughout recording,cumulative_drift Cumulative change in spike depth throughout recording,contamination_rate
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,0,7.57,3.44492,1.0,nan,None,0.0028729,37.12,0.0687267,4.79221,0.872,0.0015,nan,nan,nan,nan
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,1,48.1133,2.64645,1.0,nan,None,0.00093204,40.1523,0.102257,4.47733,0.8475,0.002,nan,nan,nan,nan
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,2,41.6267,2.76595,1.0,nan,None,0.0101575,19.5597,0.59708,3.87349,0.7235,0.00055,nan,nan,nan,nan
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,3,15.58,3.13084,1.0,nan,None,0.00681746,43.4827,0.0623582,4.5024,0.8515,0.0025,nan,nan,nan,nan
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,4,49.3067,2.56582,1.0,nan,None,0.00606356,31.5925,0.111974,4.57776,0.8575,0.001125,nan,nan,nan,nan
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,5,14.7,3.01465,1.0,nan,None,0.0128314,31.5654,0.126114,4.48018,0.8355,0.000611111,nan,nan,nan,nan
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,6,37.34,2.30398,1.0,nan,None,0.0103203,12.283,1.77468,1.06425,0.262,0.114417,nan,nan,nan,nan
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,7,63.5467,2.56239,1.0,nan,None,0.00152424,25.7088,0.240307,4.17461,0.781,0.0029,nan,nan,nan,nan
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,8,26.4567,2.93525,1.0,nan,None,0.010423,31.2902,0.15173,4.32882,0.8095,0.0013,nan,nan,nan,nan
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,0,9,4.49333,3.67005,1.0,nan,None,0.00915086,22.4424,0.378531,4.30065,0.7675,0.0004,nan,nan,nan,nan
